<a href="https://colab.research.google.com/github/deep-son/California-Housing-Price-Prediction/blob/master/California_Housing_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing 



In [ ]:
house_df_main = pd.read_csv('/kaggle/input/housing/housing.csv')
house_df = house_df_main.copy()



In [ ]:
house_df.head(10)

In [ ]:
house_df.describe(include='all')

In [ ]:
house_df.info()

In [ ]:
house_df.isnull().sum()

In [ ]:
house_df.fillna(house_df.mean(), inplace=True)

In [ ]:
house_df.isnull().sum()

In [ ]:
#initial Correlation
plt.figure(figsize = (16,10))
sns.heatmap(house_df.corr(),annot=True,center=0 )

In [ ]:
dic = {}

## Worst Model

In [ ]:
wmhouse_df = house_df.copy()
wmhouse_df.columns

In [ ]:
scaler = MinMaxScaler()
column_names_to_normalize = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value']
x = wmhouse_df[column_names_to_normalize].values
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = wmhouse_df.index)
wmhouse_df[column_names_to_normalize] = df_temp

In [ ]:
#worst model
wm_data = wmhouse_df.copy()
wmY = wm_data['median_house_value']
wm_data.drop(columns=['ocean_proximity','median_house_value'],inplace=True)
wmX_train, wmX_test, wmy_train, wmy_test = train_test_split(wm_data, wmY, test_size=0.2, random_state=1)
wm_data.columns


In [ ]:
model = linear_model.LinearRegression()
model.fit(wmX_train, wmy_train)

In [ ]:
wmy_pred = model.predict(wmX_test)

In [ ]:
MSE = metrics.mean_squared_error(wmy_test, wmy_pred)
RMSE = np.sqrt(metrics.mean_squared_error(wmy_test, wmy_pred))
scores = model.score(wmX_train, wmy_train)
dic['BaseModel'] = (RMSE,scores*100)
print(MSE,RMSE,scores*100)

## Exploring data

In [ ]:
house_df.hist(bins=50, figsize=(15, 15))

In [ ]:
sns.pairplot(house_df, x_vars=['housing_median_age', 'population', 'households', 'median_income'],y_vars ='median_house_value',hue = 'ocean_proximity')

In [ ]:
house_df.columns

In [ ]:
house_df['ocean_proximity'].value_counts()

In [ ]:
house_df['ocean_proximity'].value_counts().plot(kind='bar')

In [ ]:
sns.boxplot(x="ocean_proximity", y="median_house_value", data=house_df)

In [ ]:
sns.boxplot(x="ocean_proximity", y="median_income", data=house_df)

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(house_df['total_bedrooms'],color='red')
sns.distplot(house_df['total_rooms'],color='blue')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot((house_df['total_rooms']/house_df['total_bedrooms']),color='green')
plt.show()

In [ ]:
house_df['room_bed'] = (house_df['total_rooms']/house_df['total_bedrooms'])

In [ ]:
house_df['room_bed']

In [ ]:
house_df1 = house_df.copy()
house_df1 = house_df1.loc[ house_df1['room_bed']<10.0]

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(house_df['room_bed'],color='green',)

plt.figure(figsize=(10,6))
sns.boxplot(house_df1['room_bed'],color='red')

plt.show()

In [ ]:
house_df1.info()

In [ ]:

house_df1.plot.scatter(x='housing_median_age', y='population')


In [ ]:
house_df1 = house_df1.loc[ house_df['population']<20000]

In [ ]:
house_df1.info()

In [ ]:
house_df1.plot.scatter(x='housing_median_age', y='population')

In [ ]:
house_df1.columns

In [ ]:
house_df1['house_pop'] = house_df1['households'] / house_df1['population']

In [ ]:
house_df1.hist(bins=50, figsize=(15, 15))

In [ ]:
sns.pairplot(house_df1, x_vars=['housing_median_age', 'population', 'households', 'median_income',
       'room_bed','house_pop'],y_vars ='median_house_value')

In [ ]:
house_df1 = pd.get_dummies(house_df1,columns=['ocean_proximity'])


In [ ]:
house_df1.columns
house_df1_norm = house_df1.copy()

In [ ]:
scaler = MinMaxScaler()
column_names_to_normalize = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'room_bed', 'house_pop']
x = house_df1_norm[column_names_to_normalize].values
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = house_df1_norm.index)
house_df1_norm[column_names_to_normalize] = df_temp

In [ ]:
house_df1_norm.head()

In [ ]:
house_df1_norm.hist(bins=50,figsize=(15, 15))

In [ ]:
#test model 1
m1_data = house_df1.copy()
m1Y = m1_data['median_house_value']
m2_data = house_df1.copy()
m2_data.drop(columns=['total_rooms',
       'total_bedrooms','population','households','ocean_proximity_ISLAND'],inplace=True)
m1_data.drop(columns=['median_house_value','total_rooms',
       'total_bedrooms','population','households','ocean_proximity_ISLAND'],inplace=True)
m1_data.columns


In [ ]:
m1X_train, m1X_test, m1y_train, m1y_test = train_test_split(m1_data, m1Y, test_size=0.2,random_state=1)

In [ ]:
model1 = linear_model.LinearRegression()
model1.fit(m1X_train, m1y_train)

In [ ]:
m1y_pred = model1.predict(m1X_test)
# m1y_pred_train = model1.predict(m1X_train)

In [ ]:
print(metrics.mean_squared_error(m1y_test, m1y_pred))
print(np.sqrt(metrics.mean_squared_error(m1y_test, m1y_pred)))

In [ ]:
pd.DataFrame(zip(m1_data.columns,model1.coef_))

In [ ]:
pd.DataFrame(zip(m1y_test,m1y_pred),columns=['True','Predicted'])

In [ ]:
model1.score(m1X_train, m1y_train)

In [ ]:
m2_data.rename(columns={'ocean_proximity_<1H OCEAN': 'ocean_proximity1', 'ocean_proximity_NEAR BAY': 'ocean_proximity2', 'ocean_proximity_NEAR OCEAN': 'ocean_proximity3'}, inplace=True)

In [ ]:
lm2 = smf.ols(formula='median_house_value ~ longitude + latitude + housing_median_age  + median_income + room_bed + house_pop + ocean_proximity1 + ocean_proximity_INLAND + ocean_proximity2 + ocean_proximity3', data=m2_data).fit()

In [ ]:
lm2.summary()

## Training models

### Decision Tree

In [ ]:
model_dt = make_pipeline(preprocessing.StandardScaler(),DecisionTreeRegressor(random_state=0))
scores = cross_val_score(model_dt, m1X_train, m1y_train, cv=10)
model_dt.fit(m1X_train, m1y_train)
pred = model_dt.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['DecisionTree'] = (RMSE,scores.mean() *100)

print(MSE,RMSE)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

### Linear Regression

In [ ]:
model_lr = make_pipeline(preprocessing.StandardScaler(),linear_model.LinearRegression())
scores = cross_val_score(model_lr, m1X_train, m1y_train, cv=10)

model_lr.fit(m1X_train, m1y_train)
pred = model_lr.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['LinearRegression'] = (RMSE,scores.mean() *100)

print(MSE,RMSE)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

#### Ridge Regression

In [ ]:
ridge_reg = linear_model.Ridge()
params_Ridge = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20] , "fit_intercept": [True, False], "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
Ridge_GS = GridSearchCV(ridge_reg, param_grid=params_Ridge, n_jobs=-1)
Ridge_GS.fit(m1X_train, m1y_train)
print(Ridge_GS.best_params_)
model_rr = make_pipeline(preprocessing.StandardScaler(),linear_model.Ridge(random_state=0, **Ridge_GS.best_params_))
scores = cross_val_score(model_rr, m1X_train, m1y_train, cv=10)

model_rr.fit(m1X_train, m1y_train)
pred = model_rr.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))


dic['RidgeRegression'] = (RMSE,scores.mean() *100)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

#### Lasso Regression

In [ ]:
lasso_reg = linear_model.Lasso()
params_Lasso = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20] , "fit_intercept": [True, False]}
Lasso_GS = GridSearchCV(lasso_reg, param_grid=params_Lasso, n_jobs=-1)
Lasso_GS.fit(m1X_train, m1y_train)
print(Lasso_GS.best_params_)
model_lasso = make_pipeline(preprocessing.StandardScaler(),linear_model.Lasso(random_state=0, **Lasso_GS.best_params_, max_iter=1e+5))
scores = cross_val_score(model_lasso, m1X_train, m1y_train, cv=10)

model_lasso.fit(m1X_train, m1y_train)
pred = model_lasso.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))


dic['LassoRegression'] = (RMSE,scores.mean() *100)
print(MSE,RMSE)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

### Random Forest

In [ ]:
model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=2, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)
model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['RFD_2'] = (RMSE,scores.mean() *100)
print("Depth:2 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=5, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)


model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['RFD_5'] = (RMSE,scores.mean() *100)

print("Depth:5 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=10, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)

model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['RFD_10'] = (RMSE,scores.mean() *100)

print("Depth:10 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))

model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=20, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)

model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['RFD_20'] = (RMSE,scores.mean() *100)

print("Depth:20 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))


model_rf = make_pipeline(preprocessing.StandardScaler(),RandomForestRegressor(max_depth=50, random_state=0))
scores = cross_val_score(model_rf, m1X_train, m1y_train, cv=10)

model_rf.fit(m1X_train, m1y_train)
pred = model_rf.predict(m1X_test)

MSE = metrics.mean_squared_error(m1y_test, pred)
RMSE = np.sqrt(metrics.mean_squared_error(m1y_test, pred))

dic['RFD_50'] = (RMSE,scores.mean() *100)

print("Depth:50 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() *100, scores.std() * 2))


In [ ]:
test = model_rf.fit(m1X_train, m1y_train)

In [ ]:
pred = test.predict(m1X_test)

In [ ]:
print(np.sqrt(metrics.mean_squared_error(m1y_test, pred)))

## Final Output Comparison

In [ ]:
result = pd.DataFrame(zip(m1y_test,pred),columns=['True','Predicted'])

In [ ]:
result

In [ ]:
accuracy_model = pd.DataFrame(dic,index=['RMSE','Training_Accuracy'])
accuracy_model = accuracy_model.T
accuracy_model

In [ ]:
accuracy_model.sort_values(by=['RMSE'],inplace=True)

In [ ]:
from matplotlib import pyplot
fig, ax = pyplot.subplots(figsize=(15,8))
sns.barplot(x=accuracy_model.index,y='RMSE',data=accuracy_model,ax=ax,palette='Greens')



In [ ]:
accuracy_model.sort_values(by=['Training_Accuracy'],inplace=True)

In [ ]:
fig1, ax1 = pyplot.subplots(figsize=(15,8))
sns.barplot(x=accuracy_model.index,y='Training_Accuracy',data=accuracy_model,ax=ax1,palette='Blues')